In [3]:
# %% [markdown]
# ### **Passo 1: Importar Bibliotecas e Definir Funções**
# %%
import CoolProp.CoolProp as CP
import pandas as pd

def obter_propriedades(P, T=None, x=None, s=None):
    """Retorna h (kJ/kg) e s (kJ/kg·K) para um dado estado."""
    if T is not None:  # Estado definido por P e T
        h = CP.PropsSI('H', 'P', P, 'T', T, 'Water') / 1000  # kJ/kg
        s = CP.PropsSI('S', 'P', P, 'T', T, 'Water') / 1000  # kJ/kg·K
    elif x is not None:  # Estado definido por P e título
        h = CP.PropsSI('H', 'P', P, 'Q', x, 'Water') / 1000
        s = CP.PropsSI('S', 'P', P, 'Q', x, 'Water') / 1000
    elif s is not None:  # Estado definido por P e entropia (processo isentrópico)
        try:
            h = CP.PropsSI('H', 'P', P, 'S', s*1000, 'Water') / 1000  # s em kJ/kg·K → J/kg·K
        except:
            # Calcular título se for mistura
            s_liq = CP.PropsSI('S', 'P', P, 'Q', 0, 'Water') / 1000
            s_vap = CP.PropsSI('S', 'P', P, 'Q', 1, 'Water') / 1000
            x = (s - s_liq) / (s_vap - s_liq)
            h = CP.PropsSI('H', 'P', P, 'Q', x, 'Water') / 1000
    return h, s

# %% [markdown]
# ### **Passo 2: Criar Tabela de Propriedades**
# %%
colunas = ['Ponto', 'Pressão (kPa)', 'Temperatura (°C)', 'Entalpia (kJ/kg)', 'Entropia (kJ/kg·K)', 'Estado']
df = pd.DataFrame(columns=colunas)

# %% [markdown]
# ### **Passo 3: Calcular Propriedades para Cada Ponto**

# %% [markdown]
# **Ponto 1 (Corrente 1): Saída da Caldeira 1**
# %%
P1 = 15000  # kPa
T1 = 600     # °C
h1, s1 = obter_propriedades(P1*1e3, T=T1+273.15)
df.loc[len(df)] = [1, P1, T1, h1, s1, 'Vapor superaquecido']

# %% [markdown]
# **Ponto 2 (Corrente 2): Saída da Turbina 1 (Expansão Isentrópica)**
# %%
P2 = 4000  # kPa
h2, s2 = obter_propriedades(P2*1e3, s=s1)  # s2 = s1
T2 = CP.PropsSI('T', 'P', P2*1e3, 'H', h2*1000, 'Water') - 273.15
df.loc[len(df)] = [2, P2, round(T2,1), h2, s2, 'Mistura' if 0 < (h2 - CP.PropsSI('H', 'P', P2*1e3, 'Q', 0, 'Water')/1000) < (CP.PropsSI('H', 'P', P2*1e3, 'Q', 1, 'Water')/1000 - CP.PropsSI('H', 'P', P2*1e3, 'Q', 0, 'Water')/1000) else 'Vapor']

# %% [markdown]
# **Ponto 3 (Corrente 3): Entrada na Caldeira 2 (Regeneração)**
# %%
P3 = P2  # Mesma pressão
m3 = 0.82 * 10000  # 8200 kg/h (82% da corrente 2)
# Estado igual ao Ponto 2 (extração para regeneração)
df.loc[len(df)] = [3, P3, df.loc[df['Ponto'] == 2, 'Temperatura (°C)'].values[0],
                   df.loc[df['Ponto'] == 2, 'Entalpia (kJ/kg)'].values[0],
                   df.loc[df['Ponto'] == 2, 'Entropia (kJ/kg·K)'].values[0],
                   'Extraído para regeneração']

# %% [markdown]
# **Ponto 5 (Corrente 5): Saída da Caldeira 2 (Após Reaquecimento)**
# %%
P5 = 4000  # kPa (errata: temperatura corrigida para Ponto 5)
T5 = 600   # °C
h5, s5 = obter_propriedades(P5*1e3, T=T5+273.15)
df.loc[len(df)] = [5, P5, T5, h5, s5, 'Vapor superaquecido']

# %% [markdown]
# **Ponto 6 (Corrente 6): Saída da Turbina 2 (Expansão Isentrópica)**
# %%
P6 = 500  # kPa
h6, s6 = obter_propriedades(P6*1e3, s=s5)  # s6 = s5
T6 = CP.PropsSI('T', 'P', P6*1e3, 'H', h6*1000, 'Water') - 273.15
df.loc[len(df)] = [6, P6, round(T6,1), h6, s6, 'Mistura']

# %% [markdown]
# **Ponto 7 (Corrente 7): Extração para Regeneração (13% da Corrente 6)**
# %%
P7 = P6  # Mesma pressão
m7 = 0.13 * m3  # 1066 kg/h
df.loc[len(df)] = [7, P7, df.loc[df['Ponto'] == 6, 'Temperatura (°C)'].values[0],
                   df.loc[df['Ponto'] == 6, 'Entalpia (kJ/kg)'].values[0],
                   df.loc[df['Ponto'] == 6, 'Entropia (kJ/kg·K)'].values[0],
                   'Extraído para regeneração']

# %% [markdown]
# **Ponto 9 (Corrente 9): Entrada no Condensador**
# %%
P9 = 10  # kPa
h9 = h6  # Assume-se que não há perda de pressão
T9 = CP.PropsSI('T', 'P', P9*1e3, 'H', h9*1000, 'Water') - 273.15
df.loc[len(df)] = [9, P9, round(T9,1), h9, s6, 'Mistura']

# %% [markdown]
# **Ponto 10 (Corrente 10): Saída do Condensador (Líquido Saturado)**
# %%
P10 = P9
h10 = CP.PropsSI('H', 'P', P10*1e3, 'Q', 0, 'Water') / 1000
s10 = CP.PropsSI('S', 'P', P10*1e3, 'Q', 0, 'Water') / 1000
T10 = CP.PropsSI('T', 'P', P10*1e3, 'Q', 0, 'Water') - 273.15
df.loc[len(df)] = [10, P10, round(T10,1), h10, s10, 'Líquido saturado']

# %% [markdown]
# **Ponto 11 (Corrente 11): Saída da Bomba 1**
# %%
P11 = 500  # kPa (pressão aumentada para regeneração)
h11 = h10 + (P11*1e3 - P10*1e3)/(1000 * 1000)  # Trabalho da bomba simplificado
m9 = m3 - m7  # 7134 kg/h
W_bomba1 = (h11 - h10) * m9  # m9 = m3 - m7 = 8200 - 1066 = 7134 kg/h
T11 = CP.PropsSI('T', 'P', P11*1e3, 'H', h11*1000, 'Water') - 273.15
df.loc[len(df)] = [11, P11, round(T11,1), h11, s10, 'Líquido comprimido']

# %% [markdown]
# **Ponto 13 (Corrente 13): Saída da Bomba 2**
# %%
P13 = 15000  # kPa
h13 = h10 + (P13*1e3 - P11*1e3)/(1000 * 1000)  # Trabalho da bomba simplificado
T13 = CP.PropsSI('T', 'P', P13*1e3, 'H', h13*1000, 'Water') - 273.15
df.loc[len(df)] = [13, P13, round(T13,1), h13, s10, 'Líquido comprimido']

# %% [markdown]
# **Ponto 15 (Corrente 15): Líquido Saturado após Regeneração**
# %%
P15 = P13
h15 = CP.PropsSI('H', 'P', P15*1e3, 'Q', 0, 'Water') / 1000
s15 = CP.PropsSI('S', 'P', P15*1e3, 'Q', 0, 'Water') / 1000
df.loc[len(df)] = [15, P15, round(CP.PropsSI('T', 'P', P15*1e3, 'Q', 0, 'Water') -273.15,1), h15, s15, 'Líquido saturado']

# %% [markdown]
# ### **Passo 4: Exibir Tabela Completa**
# %%
df.set_index('Ponto', inplace=True)
df
# %% [markdown]
# ### **Passo 5: Exportar Tabela**
# %%
# df.to_csv('propriedades_ciclo_rankine_modificado.csv')

,Pressão (kPa),Temperatura (°C),Entalpia (kJ/kg),Entropia (kJ/kg·K),Estado
Ponto,,,,,
1,15000,600.0,3583.131761,6.679590,Vapor superaquecido
2,4000,374.7,3153.874543,6.679590,Vapor
3,4000,374.7,3153.874543,6.679590,Extraído para regeneração
5,4000,600.0,3674.867523,7.370528,Vapor superaquecido
6,500,275.4,3013.589893,7.370528,Mistura
7,500,275.4,3013.589893,7.370528,Extraído para regeneração
9,10,268.3,3013.589893,7.370528,Mistura
10,10,45.8,191.805945,0.649196,Líquido saturado
11,500,45.8,192.295945,0.649196,Líquido comprimido
